In [9]:
import numpy as np
import pandas as pd
from lifetimes import BetaGeoBetaBinomFitter
from lifetimes.datasets import load_donations, load_cdnow_summary_data_with_monetary_value

import pymc as pm
from pymc_marketing.clv.distributions import BetaGeoBetaBinom

In [10]:
T_true = 6
alpha_true = 1.204
beta_true = 0.750
gamma_true = 0.657
delta_true = 2.783

with pm.Model():
    alpha = pm.Normal(name="alpha", mu=alpha_true, sigma=1e-3)
    beta = pm.Normal(name="beta", mu=beta_true, sigma=1e-3)
    gamma = pm.Normal(name="gamma", mu=gamma_true, sigma=1e-3)
    delta = pm.Normal(name="delta", mu=delta_true, sigma=1e-3)

    T = pm.MutableData(name="T", value=np.array(T_true))

    BetaGeoBetaBinom(
        name="beta_geo_beta_binom",
        alpha=alpha,
        beta=beta,
        gamma=gamma,
        delta=delta,
        T=T,
    )
    prior = pm.sample_prior_predictive(samples=10)

# recency = prior["prior"]["beta_geo_beta_binom"][0][:,0]
# #frequency = prior["prior"]["beta_geo_beta_binom"][1]
# recency.mean()
prior

Sampling: [alpha, beta, beta_geo_beta_binom, delta, gamma]


Inference data with groups:
	> prior
	> constant_data

In [11]:
value = np.array([[1.5, 1], [5.3, 4], [6, 2]])
alpha = 0.55
beta = 10.58
gamma = 0.61
delta = 11.67
T = 12

BetaGeoBetaBinomFitter._loglikelihood((alpha, beta, gamma, delta), value[..., 1], value[..., 0], T)

array([ -3.94031398, -10.25427751,  -6.82582822])

In [12]:
discrete_noncontract_df = load_donations()

periods = 6
bgbb = BetaGeoBetaBinomFitter().fit(discrete_noncontract_df['frequency'].values,
                             discrete_noncontract_df['recency'].values,
                             discrete_noncontract_df['periods'].values,
                             discrete_noncontract_df['weights'].values)
bgbb

<lifetimes.BetaGeoBetaBinomFitter: fitted with 22 subjects, alpha: 1.20, beta: 0.75, delta: 2.78, gamma: 0.66>

In [13]:
method_list = [method for method in dir(BetaGeoBetaBinomFitter) if not method.startswith('_')]
print(method_list)

['conditional_expected_number_of_purchases_up_to_time', 'conditional_probability_alive', 'expected_number_of_transactions_in_first_n_periods', 'fit', 'load_model', 'save_model', 'summary']


In [14]:
# equation 13 in paper
bgbb.conditional_expected_number_of_purchases_up_to_time(5,
    discrete_noncontract_df['frequency'],
    discrete_noncontract_df['recency'],
    discrete_noncontract_df['periods'])

0     0.072863
1     0.085696
2     0.314238
3     0.593853
4     0.839396
5     1.021689
6     1.147885
7     0.119121
8     0.536111
9     1.057604
10    1.443042
11    1.668817
12    0.223595
13    1.034572
14    1.804703
15    2.189749
16    0.583192
17    2.030024
18    2.710681
19    1.812942
20    3.231612
21    3.752544
dtype: float64

In [15]:
# equation 11 in paper
bgbb.conditional_probability_alive(10,
    discrete_noncontract_df['frequency'],
    discrete_noncontract_df['recency'],
    discrete_noncontract_df['periods'])

0     0.070072
1     0.045012
2     0.165056
3     0.311927
4     0.440900
5     0.536651
6     0.602936
7     0.043038
8     0.193695
9     0.382108
10    0.521365
11    0.602936
12    0.061566
13    0.284864
14    0.496916
15    0.602936
16    0.129719
17    0.451538
18    0.602936
19    0.338249
20    0.602936
21    0.602936
dtype: float64

In [24]:
# TODO: write and test (8) as a replacement. Compare against just aggregating means across the exploded DF 
# TODO: Can the arviz functions in the BetaGeoBetaBinom distribution block preclude the need for this?
# TODO: Replace this with (9) or (10) in a future PR, since that expression can predict interval ranges

# equation 7 in paper, but that's for probabilities. should it be 8 for predicting mean n?
# yeah, this function should be renamed for clarity. 
# it distributes customers in the dataset across n transaction opportunies
# it works better as an evaluation function, since it assumes a fixed customer population size
# if n > n_periods, it will keep right on predicting. This may be a bug
bgbb.expected_number_of_transactions_in_first_n_periods(n=6)

,model
frequency,
0,3454.881979
1,1888.654343
2,1348.882330
3,1113.378533
4,1017.922413
5,1027.166049
6,1253.114353


### Expected transactions in N periods
This expression will blow up to inf with large values of n (n=167 in this example). Recalculating on the log scale will allow for larger values, but this isn't possible if gamma < 1 because term1 will be negative.

In [8]:
from scipy import special
from numpy import log,exp

n = 5
alpha,beta,delta,gamma = bgbb._unload_params('alpha','beta','delta','gamma')

# add a larger gamma value for testing
#gamma = .9

log_scale = False

if not log_scale:
    term1 = alpha/(alpha+beta)*delta/(gamma-1)
    term2 = 1-(special.gamma(gamma+delta))/special.gamma(gamma+delta+n)*(special.gamma(1+delta+n))/special.gamma(1+delta)
    expected_purchases_n_periods = term1 * term2
else:
    term1 = log(alpha/(alpha+beta)) + log(delta/(gamma-1))
    term2 = special.gammaln(gamma+delta) - special.gammaln(gamma+delta+n) + special.gammaln(1+delta+n) - special.gammaln(1+delta)
    expected_purchases_n_periods = exp(term1) - exp(term2)

print(f'average of {expected_purchases_n_periods} purchases expected in {n} opportunities')

average of 1.938137499995133 purchases expected in 5 opportunities


In [17]:
bgbb._unload_params('alpha','beta','delta','gamma')

[1.203522393608101, 0.7497163581757842, 2.783441982887136, 0.6567181695498788]